# Formulae preparation

In [1]:
import spot as s
import sys

In this notebook we generate files with formulae that we further use for benchmarking Seminator. We use three types of formulae regarding their source. These are

1. **classical benchmarking formulae**,
2. **random formulae**, and
3. **`ltl2ldba` benchmarking sets**.

## Formulae from literature

* beem.ltl: R. Pelánek: _BEEM: Benchmarks for explicit model checkers._ Proceedings of Spin'07.  LNCS 4595.
* liberouter.ltl: Holeček, T. Kratochvila, V. Řehák, D. Šafránek, and P. Šimeček: _Verification  Results  in  Liberouter Project._  Tech. Report 03, CESNET, 2004.
* dac: M.  B.  Dwyer and G. S. Avrunin and J. C. Corbett: _Property Specification Patterns for Finite-state Verification._ Proceedings of FMSP'98.
* eh: K. Etessami and G. J. Holzmann: _Optimizing Büchi Automata._  Proceedings of Concur'00.  LNCS 1877.
* sb: F. Somenzi and R. Bloem: _Efficient Büchi Automata for LTL Formulae._  Proceedings of CAV'00.  LNCS 1855.

There are many duplicates among the sources mentioned above. Therefore, we process them to minimize the level of duplicates. The steps are:
1. The temporal operators $\mathsf{W}$ and $\mathsf{M}$ are removed by applying standard equivalences.
1. Atomic propositions are renamed in alphabetical order and also free boolean combinations of AP are replaced by fresh AP. For example, the formula $\mathsf{G}(a \lor b) \land \mathsf{G} c$ is relabelled to $\mathsf{G} a \land \mathsf{G} b$.
2. Formulae are simplified by SPOT's reductions techniques.
3. Formulae equivallent to `True` or `False` are removed.

Next, we add also a negations of already chosen formulae.

In [2]:
beem_file     = 'formulae/beem.ltl'
lib_file      = 'formulae/liberouter.ltl'
tmp_file      = 'formulae/tmp.ltl'
lit_pref      = 'formulae/literature'
lit_file      = lit_pref + '.ltl'
rand_pref     = 'formulae/random'

In [35]:
!genltl --dac-patterns --eh-patterns --sb-patterns | \
ltlfilt \
    $beem_file \
    $lib_file  \
    -F -  > $tmp_file
!ltlfilt $tmp_file -c

172


In [37]:
!ltlfilt --negate $tmp_file | \
ltlfilt $tmp_file -F - --unique -r3 --remove-wm --relabel-bool=abc | \
     ltlfilt -v --equivalent-to=0 | ltlfilt -v --equivalent-to=1> $lit_file
!ltlfilt -c $lit_file

222


In [38]:
def is_det(form):
    aut = s.translate(form,'Deterministic')
    return s.is_deterministic(aut)

In [39]:
def is_sd(form):
    aut = s.translate(form,'Deterministic')
    return s.is_semi_deterministic(aut)

In [1]:
def is_cd(form):
    res = !ltl2tgba -f '$form' | ./seminator --is-cd
    return res[0] == '1'

In [41]:
def categorize(form_file,prefix):
    with open(form_file,'r') as source,\
         open('{}_det.ltl'.format(prefix),'w') as det,\
         open('{}_cd.ltl'.format(prefix),'w') as cd,\
         open('{}_sd.ltl'.format(prefix),'w') as sd,\
         open('{}_nd.ltl'.format(prefix),'w') as nd:
        for form in source:
            if is_det(form):
                print(form,file=det,end='')
            elif is_cd(form):
                print(form,file=cd,end='')
            elif is_sd(form):
                print(form,file=sd,end='')
            else:
                print(form,file=nd,end='')

In [42]:
categorize(lit_file,lit_pref)

### The number of formulae that can be translated to a det. TGBA

In [43]:
for t in ['det','cd','sd','nd']:
    count = sum(1 for line in open('{}_{}.ltl'.format(lit_pref,t)))
    print('Automata of type {}:\t{}'.format(t,count))

Automata of type det:	148
Automata of type cd:	47
Automata of type sd:	3
Automata of type nd:	24


## Random formulae

In [13]:
def generate(n=100,func=(lambda x: True),filename=None,seed=0):
    '''Generates a specified number of formulas that are
    accepted by the given function. If filename specified
    it prints them into the given file.
    '''
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = s.randltl(['a','b','c','d'],ltl_priorities='M=0,W=0,xor=0',
                  simplify=3,tree_size=15,seed=seed).\
                  relabel_bse(s.Abc).unabbreviate('WM')
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        #form = spot.negative_normal_form(form)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

generate 100 formulas for each type of TGBA produced by spot

In [25]:
for t,f in [('det',is_det),\
            ('cd',(lambda x: (not is_det(x)) and is_cd(x))),\
            ('sd',(lambda x: (not is_cd(x)) and is_sd(x))),\
            ('nd',(lambda x: not is_sd(x)))]:
    generate(100,f,'{}_{}.ltl'.format(rand_pref,t))

In [26]:
for t in ['det','cd','sd','nd']:
    count = sum(1 for line in open('{}_{}.ltl'.format(rand_pref,t)))
    print('Automata of type {}:\t{}'.format(t,count))

Automata of type det:	100
Automata of type cd:	100
Automata of type sd:	100
Automata of type nd:	100


## ltl2ldba benchmark set -- not included in the LPAR paper

The benchmark set used in _Salomon Sickert, Javier Esparza, Stefan Jaax, Jan Kretínský:
**Limit-Deterministic Büchi Automata for Linear Temporal Logic.** CAV (2) 2016: 312-332_ consists of five types of formulae.

1. Formulae already used for benchmarking Raninizer 1 
$$\psi_j = \bigwedge_{0<=i<j} \mathsf{G}\mathsf{F}a_i \implies \bigwedge_{0<=i<j} \mathsf{G}\mathsf{F}b_i$$
2. Fairness constraints 
$$ \Psi_k = \bigwedge_{0<=i<k} \big(\mathsf{G}\mathsf{F}a_i \lor \mathsf{F}\mathsf{G}b_i\big)$$
3. Formulae with light nesting of modal operators
    $$ϕ1 = \mathsf{G}\mathsf{F}(\mathsf{F}a ∨ \mathsf{G}b ∨ \mathsf{F}\mathsf{G}(a ∨ (\mathsf{X}b)), \\
    ϕ2 = \mathsf{F}\mathsf{G}(\mathsf{G}a ∨ \mathsf{F}¬b ∨ \mathsf{G}\mathsf{F}(a ∧ \mathsf{X}b)), \\
    ϕ3 = \mathsf{G}\mathsf{F}(\mathsf{F}a ∨ \mathsf{G}\mathsf{X}b ∨ \mathsf{F}\mathsf{G}(a ∨ \mathsf{X}\mathsf{X}b))$$,
4. Formulae with deep nesting modal operators 
$$ f(0, j) = (\mathsf{G}\mathsf{F}a_0)\,\mathsf{U}\,(\mathsf{X}^jb)\\
f(i + 1, j) = (\mathsf{G}\mathsf{F}a_{i+1})\,\mathsf{U}\,(\mathsf{G}\,f(i, j))
$$
5. Formulae from _Frantisek Blahoudek, Mojmír Kretínský, Jan Strejcek:
**Comparison of LTL to Deterministic Rabin Automata Translators.** LPAR 2013: 164-172_. We do not include them here because they mostly overlap with the classical and random formulae from above.

### Functions to generate the formulae

### Writing formulae to files